Import

In [72]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

document_info_index = 0

vulnerability_db = {"VulnerabilityID":[],"DocumentIDFK":[], "Ordinal":[], "CVE":[], "Title":[]}
vulnerability_db_status = {"VulnerabilityFK":[], "StatusType": [], "ProductID":[]}
vulnerability_db_notes = {"VulnerabilityFK":[], "NotesTitle":[], "NotesType":[], "NotesOrdinal":[], "Note":[]}
vulnerability_db_threats = {"VulnerabilityFK":[], "TreatsType":[], "Description":[], "ProductID":[]}
vulnerability_db_score_set = {"VulnerabilityFK":[],"BaseScore":[], "TemporalScore":[], "Vector":[], 'ProductID':[]}
vulnerability_db_acknowledgment= {"VulnerabilityFK":[], "Name":[], "URL":[]}
vulnerability_db_revision = {"VulnerabilityFK":[], "Number":[], 'Date':[], 'Description':[]}

productdb = {'ProductID':[], 'ProductName':[], 'productdbType':[], 'productdbName':[]}

notes_db = {"DocumentIDFK":[], "NoteID": [], "notes_dbTitle":[], "notes_dbAudience":[], "notes_dbType":[], "Ordinal":[]}

document_info_db = {"DocumentID":[], "ID":[], "Alias":[], 'Status':[], "Version":[], "RevisionHistoryNumber":[], "RevisionHistoryDate":[], 
                    "RevisionHistoryDescription":[], "InitialReleaseDate":[], "CurrentReleaseDate":[], 'Pubishertype':[], 'ContactDetails':[], 'IssuingAuthority':[], 
                    'DocumentTitle':[], 'DocumentType':[], 'vuln':[],'dc':[],'cvrf-common':[],'prod':[],'scap-core':[],'cvssv2':[],'cpe-lang':[],'sch':[],'cvrf':[]}


#ProductTree

In [73]:
def product_tree(soup, productdb):
    library = soup.find('ProductTree')

    def process_node(node, productdb):

        if node.name is not None:
            if 'ProductID' in node.attrs:
                productdb['ProductID'].append(node.attrs['ProductID'])
                productdb['ProductName'].append(node.text)
            
            if 'Type' not in node.attrs and node.name != 'ProductTree':
                if 'Type' in node.parent.attrs:
                    productdb['productdbType'].append(node.parent.attrs['Type'])
                    productdb['productdbName'].append(node.parent.attrs['Name'])
                else:
                    productdb['productdbType'].append(None)
                    productdb['productdbName'].append(None)

            for child in node.children:
                process_node(child, productdb)


    process_node(library, productdb)



#Vulnerability

In [74]:

def Get_Vulnerability(soup, vulnerability_db, vulnerability_db_status, vulnerability_db_notes, vulnerability_db_threats, vulnerability_db_score_set,vulnerability_db_acknowledgment, vulnerability_db_revision):
    def process_vulnerability(node, vulnerability_db, vulnerability_index):
        vulnerability_db['Ordinal'].append(node.get('Ordinal'))
        vulnerability_db['Title'].append(node.find('Title').text if node.find('Title').text else None)
        vulnerability_db['CVE'].append(node.find('CVE').text)
        vulnerability_db['VulnerabilityID'].append(vulnerability_index+1)

    def process_status(node, vulnerability_db_status, vulnerability_index):
        vulnerability_db_status['ProductID'].append(node.text)
        vulnerability_db_status['StatusType'].append(node.parent.get('Type'))
        vulnerability_db_status['VulnerabilityFK'].append(vulnerability_index)

    def process_notes(node, vulnerability_db_notes, vulnerability_index):
        vulnerability_db_notes['VulnerabilityFK'].append(vulnerability_index)
        vulnerability_db_notes['NotesTitle'].append(node.get('Title'))
        vulnerability_db_notes['NotesType'].append(node.get('Type'))
        vulnerability_db_notes['NotesOrdinal'].append(node.get('Ordinal'))
        vulnerability_db_notes['Note'].append(node.text)

    def process_threats(node, vulnerability_db_threats, vulnerability_index):
        vulnerability_db_threats['VulnerabilityFK'].append(vulnerability_index)
        vulnerability_db_threats['TreatsType'].append(node.get('Type'))
        product_id = node.find('ProductID')
        vulnerability_db_threats['ProductID'].append(product_id.text if product_id else None)
        description = node.find('Description')
        vulnerability_db_threats['Description'].append(description.text if description else None)

    def process_score_set(node, vulnerability_db_score_set, vulnerability_index):
        vulnerability_db_score_set['VulnerabilityFK'].append(vulnerability_index)
        vulnerability_db_score_set['BaseScore'].append(node.find('BaseScore').text)
        vulnerability_db_score_set['TemporalScore'].append(node.find('TemporalScore').text)
        vulnerability_db_score_set['Vector'].append(node.find('Vector').text)
        vulnerability_db_score_set['ProductID'].append(node.find('ProductID').text)

    def process_acknowledgment(node, vulnerability_db_acknowledgment, vulnerability_index):
        vulnerability_db_acknowledgment['VulnerabilityFK'].append(vulnerability_index)
        name = node.find('Name')
        vulnerability_db_acknowledgment['Name'].append(name.text if name and name.text else None)
        url = node.find('URL')
        vulnerability_db_acknowledgment['URL'].append(url.text if url and url.text else None)

    def process_revision(node, vulnerability_db_revision, vulnerability_index):
        vulnerability_db_revision['VulnerabilityFK'].append(vulnerability_index)
        vulnerability_db_revision['Number'].append(node.find('Number').text)
        vulnerability_db_revision['Date'].append(node.find('Date').text)
        vulnerability_db_revision['Description'].append(node.find('Description').text if node.find('Description').text else None)


    def vulnerability(node, vulnerability_db):
        if node.name is None:
            return
        
        vulnerability_index = len(vulnerability_db['Ordinal']) - 1
        

        if node.name == 'Vulnerability' and 'Ordinal' in node.attrs:
            process_vulnerability(node, vulnerability_db, vulnerability_index)
        elif node.name == 'ProductID' and node.parent.name == "Status":
            process_status(node, vulnerability_db_status, vulnerability_index)
        elif node.name == 'Note' and node.parent.name == "Notes":
            process_notes(node, vulnerability_db_notes, vulnerability_index)
        elif node.name == 'Threat' and node.parent.name == "Threats":
            process_threats(node, vulnerability_db_threats, vulnerability_index)
        elif node.name == 'ScoreSet' and node.parent.name == "CVSSScoreSets":
            process_score_set(node, vulnerability_db_score_set, vulnerability_index)
        elif node.name == 'Acknowledgment' and node.parent.name == 'Acknowledgments':
            process_acknowledgment(node, vulnerability_db_acknowledgment, vulnerability_index)
        elif node.name == 'Revision' and node.parent.name == 'RevisionHistory':
            process_revision(node, vulnerability_db_revision, vulnerability_index)

        for child in node.children:
            vulnerability(child, vulnerability_db)


    lib = soup.find('cvrfdoc').children
    for child in lib:
        if child.name == "Vulnerability":
            vulnerability_db['DocumentIDFK'].append(document_info_index)
            vulnerability(child, vulnerability_db)





#DocumentNotes

In [75]:

def Get_DocumentNotes(soup, notes_db):
    def document_notes(node, notes_db):
        if node.name is None:
            return
        notes_index = len(notes_db['notes_dbAudience'])
        if node.name == 'Note' and node.parent.name == 'DocumentNotes':
            notes_db["DocumentIDFK"].append(document_info_index)
            notes_db["notes_dbTitle"].append(node.get('Title'))
            notes_db["notes_dbAudience"].append(node.get('Audience'))
            notes_db["notes_dbType"].append(node.get('Type'))
            notes_db["Ordinal"].append(node.get('Ordinal'))
            notes_db["NoteID"].append(notes_index)
        
        for child in node.children:
            document_notes(child, notes_db) 
        

    lib = soup.find('cvrfdoc').children
    for child in lib:
        if child.name == "DocumentNotes":
            document_notes(child, notes_db)


#DocumentTracking + DocumentPublisher + DocumentType + DocumentTitle + attrs

In [76]:
def Doc_Info(soup, document_info_db):
    def documen_tracking(node,document_info_db):
        if node.name is None:
            return
        
        if node.name == "ID" and node.parent.name == "Identification":
            document_info_db["ID"].append(node.text if node.text else None)
            document_info_db['DocumentID'].append(document_info_index)
        elif node.name == "Alias" and node.parent.name == "Identification":
            document_info_db["Alias"].append(node.text if node.text else None)
        elif node.name == "Status" and node.parent.name == "DocumentTracking":
            document_info_db["Status"].append(node.text if node.text else None)
        elif node.name == "Version" and node.parent.name == "DocumentTracking":
            document_info_db["Version"].append(node.text if node.text else None)
        elif node.name == "Number" and node.parent.name == "Revision":
            document_info_db["RevisionHistoryNumber"].append(node.text if node.text else None)
        elif node.name == "Date" and node.parent.name == "Revision":
            document_info_db["RevisionHistoryDate"].append(node.text if node.text else None)
        elif node.name == "Description" and node.parent.name == "Revision":
            document_info_db["RevisionHistoryDescription"].append(node.text if node.text else None)
        elif node.name == "InitialReleaseDate" and node.parent.name == "DocumentTracking":
            document_info_db["InitialReleaseDate"].append(node.text if node.text else None)
        elif node.name == "CurrentReleaseDate" and node.parent.name == "DocumentTracking":
            document_info_db["CurrentReleaseDate"].append(node.text if node.text else None)
    
    
        for child in node.children:
            documen_tracking(child, document_info_db) 

    def documen_publisher(node,document_info_db):
        if node.name is None:
            return
        
        if node.name == 'DocumentPublisher' and node.parent.name == 'cvrfdoc':
            document_info_db['Pubishertype'].append(node.get('Type'))
        if node.name == 'ContactDetails' and node.parent.name == 'DocumentPublisher':
            document_info_db['ContactDetails'].append(node.text if node.text else None)
        if node.name == 'IssuingAuthority' and node.parent.name == 'DocumentPublisher':
            document_info_db['IssuingAuthority'].append(node.text if node.text else None)

        for child in node.children:
            documen_publisher(child, document_info_db) 

    lib = soup.find('cvrfdoc')

    document_info_db['vuln'].append(lib.get('xmlns:vuln'))
    document_info_db['dc'].append(lib.get('xmlns:dc'))
    document_info_db['cvrf-common'].append(lib.get('xmlns:cvrf-common'))
    document_info_db['scap-core'].append(lib.get('xmlns:scap-core'))
    document_info_db['prod'].append(lib.get('xmlns:prod'))
    document_info_db['cvssv2'].append(lib.get('xmlns:cvssv2'))
    document_info_db['cpe-lang'].append(lib.get('xmlns:cpe-lang'))
    document_info_db['sch'].append(lib.get('xmlns:sch'))
    document_info_db['cvrf'].append(lib.get('xmlns:cvrf'))

    lib = soup.find('cvrfdoc').children
    for child in lib:
        if child.name == "DocumentTracking":
            documen_tracking(child, document_info_db)
        if child.name == "DocumentPublisher":
            documen_publisher(child, document_info_db)
        if child.name == 'DocumentTitle':
            document_info_db['DocumentTitle'].append(child.text if child.text else None)
        if child.name == 'DocumentType':
            document_info_db['DocumentType'].append(child.text if child.text else None)



In [77]:
# doc_month_array = ["Aug","Sep",'Oct', "Nov", "Dec", "Jan", 'Feb', "Mar", "Apr", "May", "Jun", "Jul"]
# doc_year_array = ['2023', '2024']

for i in ['2023']:
    for j in ["Aug"]:
        soup = ""
        url = f'https://api.msrc.microsoft.com/cvrf/v3.0/cvrf/{i}-{j}'

        # Получаем данные по ссылке
        response = requests.get(url)

        # Проверка успешности запроса
        if response.status_code == 200:
            print(f"{i}-{j}")
            document_info_index +=1
            soup = BeautifulSoup(response.content, "xml")
            product_tree(soup, productdb)
            Get_Vulnerability(soup, vulnerability_db, vulnerability_db_status, vulnerability_db_notes, vulnerability_db_threats, vulnerability_db_score_set,vulnerability_db_acknowledgment, vulnerability_db_revision)
            Get_DocumentNotes(soup, notes_db)
            Doc_Info(soup, document_info_db)


            
        else:
            print("Не удалось получить данные, статус код:", response.status_code)
    

    

productdb = pd.DataFrame(productdb)
vulnerability_db = pd.DataFrame(vulnerability_db)
vulnerability_db_status = pd.DataFrame(vulnerability_db_status)
vulnerability_db_notes = pd.DataFrame(vulnerability_db_notes)
vulnerability_db_threats = pd.DataFrame(vulnerability_db_threats)
vulnerability_db_score_set = pd.DataFrame(vulnerability_db_score_set)
vulnerability_db_acknowledgment = pd.DataFrame(vulnerability_db_acknowledgment)
vulnerability_db_revision = pd.DataFrame(vulnerability_db_revision)
notes_db = pd.DataFrame(notes_db)
document_info_db = pd.DataFrame(document_info_db)

cve_table = vulnerability_db['CVE'].unique()
print(cve_table)
            

2023-Aug
['CVE-2023-36787' 'CVE-2023-4427' 'CVE-2023-4428' 'CVE-2023-4431'
 'CVE-2023-4430' 'CVE-2023-4429' 'CVE-2023-21709' 'CVE-2023-35368'
 'CVE-2023-35359' 'CVE-2023-36865' 'CVE-2023-36866' 'CVE-2023-36873'
 'CVE-2023-36876' 'CVE-2023-36882' 'CVE-2023-36889' 'CVE-2023-36898'
 'CVE-2023-36899' 'CVE-2023-36900' 'CVE-2023-36903' 'CVE-2023-36904'
 'CVE-2023-36905' 'CVE-2023-36906' 'CVE-2023-36907' 'CVE-2023-36908'
 'CVE-2023-36909' 'CVE-2023-36910' 'CVE-2023-36911' 'CVE-2023-36912'
 'CVE-2023-36913' 'CVE-2023-36914' 'CVE-2023-35376' 'CVE-2023-38254'
 'CVE-2023-35377' 'CVE-2023-35378' 'CVE-2023-35379' 'CVE-2023-35380'
 'CVE-2023-35381' 'CVE-2023-35382' 'CVE-2023-35383' 'CVE-2023-35384'
 'CVE-2023-35385' 'CVE-2023-35386' 'CVE-2023-35387' 'CVE-2023-35389'
 'CVE-2023-35393' 'CVE-2023-35394' 'CVE-2023-38188' 'CVE-2023-38186'
 'CVE-2023-38185' 'CVE-2023-38184' 'CVE-2023-20569' 'CVE-2023-38175'
 'CVE-2023-38172' 'CVE-2023-38170' 'CVE-2023-38169' 'CVE-2023-38167'
 'ADV230003' 'CVE-2023-38157' 

In [78]:


vulnerability_db = vulnerability_db.head(20)

vulnerability_db_status = pd.merge(vulnerability_db, vulnerability_db_status, left_on = "VulnerabilityID", right_on='VulnerabilityFK', how='inner')
vulnerability_db_score_set = pd.merge(vulnerability_db, vulnerability_db_score_set, left_on = "VulnerabilityID", right_on='VulnerabilityFK', how='inner')
vulnerability_db_acknowledgment= pd.merge(vulnerability_db, vulnerability_db_acknowledgment, left_on = "VulnerabilityID", right_on='VulnerabilityFK', how='inner')
vulnerability_db_notes= pd.merge(vulnerability_db, vulnerability_db_notes, left_on = "VulnerabilityID", right_on='VulnerabilityFK', how='inner')
vulnerability_db_threats= pd.merge(vulnerability_db, vulnerability_db_threats, left_on = "VulnerabilityID", right_on='VulnerabilityFK', how='inner')
vulnerability_db_revision= pd.merge(vulnerability_db, vulnerability_db_revision, left_on = "VulnerabilityID", right_on='VulnerabilityFK', how='inner')


vulnerability_db_status.drop(columns=['VulnerabilityFK', 'Ordinal'], inplace=True)
vulnerability_db_score_set.drop(columns=['VulnerabilityFK', 'DocumentIDFK', 'Ordinal', 'CVE', 'Title', 'ProductID'], inplace=True)
vulnerability_db_acknowledgment.drop(columns=['VulnerabilityFK', 'DocumentIDFK', 'Ordinal', 'CVE', 'Title'], inplace=True)
vulnerability_db_notes.drop(columns=['VulnerabilityFK', 'DocumentIDFK', 'CVE', 'Title', 'Ordinal'], inplace=True)
vulnerability_db_threats.drop(columns=['VulnerabilityFK', 'DocumentIDFK', 'Ordinal', 'CVE', 'Title', 'ProductID'], inplace=True)
vulnerability_db_revision.drop(columns=['VulnerabilityFK', 'DocumentIDFK', 'Ordinal', 'CVE', 'Title'],  inplace=True)

notes_db.drop(columns='Ordinal', inplace=True)

# print(vulnerability_db_score_set)

result = pd.merge(vulnerability_db_status, vulnerability_db_score_set, on= 'VulnerabilityID', how='inner')

grop_prod = result.groupby('CVE')['ProductID'].agg(lambda x: list(set(x))).reset_index()

group_var =  result.groupby('CVE')['Title'].agg(lambda x: list(set(x))[0]).reset_index()
grop_prod = pd.merge(grop_prod, group_var, on = 'CVE', how = 'inner')

group_var =  result.groupby('CVE')['StatusType'].agg(lambda x: list(set(x))[0]).reset_index()
grop_prod = pd.merge(grop_prod, group_var, on = 'CVE', how = 'inner')

group_var =  result.groupby('CVE')['BaseScore'].agg(lambda x: list(set(x))[0]).reset_index()
grop_prod = pd.merge(grop_prod, group_var, on = 'CVE', how = 'inner')

group_var =  result.groupby('CVE')['TemporalScore'].agg(lambda x: list(set(x))[0]).reset_index()
grop_prod = pd.merge(grop_prod, group_var, on = 'CVE', how = 'inner')

group_var =  result.groupby('CVE')['Vector'].agg(lambda x: list(set(x))[0]).reset_index()
grop_prod = pd.merge(grop_prod, group_var, on = 'CVE', how = 'inner')

group_var =  result.groupby('CVE')['VulnerabilityID'].agg(lambda x: list(set(x))).reset_index()
grop_prod = pd.merge(grop_prod, group_var, on = 'CVE', how = 'inner')
# result = pd.merge(result, vulnerability_db_threats, on= 'VulnerabilityID', how='inner')
# # result = result.head(256)
# result = pd.merge(result, vulnerability_db_notes, on = 'VulnerabilityID', how='inner')
# result = pd.merge(result, vulnerability_db_acknowledgment, on = 'VulnerabilityID', how='inner')
# result = pd.merge(result, vulnerability_db_revision, on= 'VulnerabilityID', how ='inner')

# result = pd.merge(result, productdb, on='ProductID', how='inner')

# result = pd.merge(result, notes_db, on = 'DocumentIDFK', how='inner')

# result = pd.merge(result, document_info_db, left_on='DocumentIDFK', right_on= 'DocumentID', how='inner')


print(grop_prod)


               CVE                                          ProductID  \
0   CVE-2023-21709                              [12039, 12038, 12191]   
1   CVE-2023-35359  [11569, 10051, 10049, 10378, 10379, 9318, 1072...   
2   CVE-2023-35368                              [12039, 12038, 12191]   
3   CVE-2023-36787                                            [11655]   
4   CVE-2023-36865         [11573, 11762, 11574, 11763, 11953, 11952]   
5   CVE-2023-36866  [11573, 11762, 10744, 11574, 11763, 11953, 107...   
6   CVE-2023-36873  [11650-10855, 11676-11569, 11676-11568, 12079-...   
7   CVE-2023-36876                                     [10051, 10049]   
8   CVE-2023-36882  [11569, 10051, 10049, 10378, 10379, 9318, 1072...   
9   CVE-2023-36889  [11569, 10051, 10049, 10378, 10379, 9318, 1072...   
10  CVE-2023-36898                       [12086, 11927, 12085, 11926]   
11  CVE-2023-36899  [11650-10855, 11676-11569, 11676-11568, 12079-...   
12  CVE-2023-36900  [11569, 10051, 10049, 10378, 10

#Match KB & CVE

In [79]:


kb_lake = pd.read_csv('Security Updates 2024-10-08-021008pm.csv')

print(kb_lake['Article'].unique())

kb_lake['Release date'] = pd.to_datetime(kb_lake['Release date'])


kb_lake = kb_lake.loc[kb_lake.groupby('Article')['Release date'].idxmax()]


kb_lake = kb_lake[kb_lake['Article'].apply(lambda x: isinstance(x, int)) | kb_lake['Article'].astype(str).str.isnumeric()]
# kb_lake.drop(columns=['Product', 'Platform'],  inplace=True)
# kb_lake['Article'] = 'KB'+ kb_lake['Article']

# kb_lake = pd.merge(kb_lake, result, left_on = "Details", right_on='CVE', how='inner')


# kb_lake.to_excel("result.xlsx")
print(kb_lake['Details'].unique())

['Release Notes' nan 'Click to Run' '5002634' '5043080' '5043055'
 '5043076' '5043064' '5043067' '5042881' '5042880' '5043138' '5043050'
 '5043051' '5043125' '5043129' '5043092' '5043135' '5043087' '5042749'
 '5042578' '5043083' '5042211' '5042215' '5042209' '5042207' '5042214'
 '5042217' '5040438' '5040442' '5043049' '5043254' '5002640' '5002639'
 '5002624' '5002566' '5042530' '5042529' '5042528' '5002605' '5002601'
 'MAU' 'XXX' '5040437' '5040430' '5041571' '5041585' '5041580' '5041592'
 '5041160' '5041773' '5041782' '5041573' '5041578' '5041851' '5041838'
 '5041823' '5041850' '5041847' '5041828' '5041557' 'grub2' '5042320'
 '5042321' '5042322' '5002561' '5040434' '5040427' '5040431' '5002626'
 '5002586' '5043126' '5039294' '5039260' '5039214' '5039225' '5039236'
 '5039212' '5039211' '5039213' '5039227' '5039330' '5039217' '5040456'
 '5040485' '5040497' '5040498' '5040499' '5040490' '5040448' '5041770'
 '5042132' '5002625' '5002570' 'openssh' '5041080' '5041017' '5041020'
 '5041016' 